In [66]:
# For compatibility across multiple platforms
import os
import numpy as np
import pandas as pd
from scipy import spatial


# Load files using DictReader in Python
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import cluster
from sklearn import preprocessing
from sklearn.model_selection import KFold
import sklearn


In [3]:
train_filepath = "pubg-finish-placement-prediction/train_V2_clean.csv"

#trainset_file = open(train_filepath,'rU')
trainset = pd.read_csv(train_filepath,index_col=0)

/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
print(len(trainset))

4446965


# SET UP FOR MODEL ONE

## Select the trainset features for tree one 

In [29]:
x_one = trainset.drop(["matchType","winPlacePerc"],axis=1)
y_one = trainset["winPlacePerc"]
x_one_train, x_one_val, y_one_train, y_one_val = train_test_split(x_one, y_one, test_size=0.2)


#set the dmatrix
trainset_one_dmatrix = xgb.DMatrix(x_one_train.values,label=y_one_train.values,feature_names=x_one.columns)
valset_one_dmatrix = xgb.DMatrix(x_one_val.values,label=y_one_val.values,feature_names=x_one.columns)

In [24]:
x_one_train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
1583144,24542524405928559,20786547937258173,69453740819561907,0,1,163.50,1,1,0,25,...,-1,0,2204.0,0,0.000,0,0,1566.0,6,1500
1028157,55031102917920668,17299994861769979,25814580387765,0,4,415.30,3,0,15,4,...,1612,0,2770.0,0,0.000,0,0,3314.0,4,0
3530203,60977608181601563,19650154976072193,19132539195260231,1,3,96.44,0,0,10,48,...,-1,0,1334.0,0,9.805,0,0,2128.0,6,1591
914973,70834705752452292,31383514614407595,10029184767505038,1,2,58.44,1,0,2,23,...,1423,1,10640.0,0,10.600,0,0,2198.0,5,0
4303837,46950122306009576,5009077649115699,41463396527259712,0,4,299.30,1,0,2,8,...,-1,1,4494.0,0,0.000,0,0,2410.0,5,1487


## Configure tree one

In [30]:
params = {
    "max_depth" : 5,
    "eval_metric" : ["mae"],
}


xgb.plot_importance(tree_one,importance_type='cover')
xgb.plot_importance(tree_one,importance_type='weight')

# SET UP FOR MODEL TWO

## Select the trainset features for tree two

In [56]:
x_two = trainset.drop(["killPoints","matchType","winPlacePerc"],axis=1)
y_two = trainset["winPlacePerc"]
x_two_train, x_two_val, y_two_train, y_two_val = train_test_split(x_two, y_two, test_size=0.2)


#set the dmatrix
trainset_two_dmatrix = xgb.DMatrix(x_two_train.values,label=y_two_train.values,feature_names=x_two.columns)
valset_two_dmatrix = xgb.DMatrix(x_two_val.values,label=y_two_val.values,feature_names=x_two.columns)

In [57]:
x_two_train.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
3140146,22323224034941961,42205670539263708,45755220675994269,0,0,69.91,0,0,0,84,...,-1,0,0.000,0,0.00,0,0,101.5,2,1475
2746801,5521130089164425,24796665285375182,29144955130336601,0,0,0.00,0,0,0,86,...,1500,0,0.000,0,0.00,0,0,0.0,0,0
2730122,39771873531777600,66889691508253690,31906770322883162,0,5,232.70,1,0,4,26,...,-1,1,0.013,0,270.80,0,0,3154.0,12,1478
3287956,54434467478264713,33522484765856452,57711492227264518,0,6,308.40,1,2,9,11,...,1473,0,3279.000,0,24.59,0,0,3216.0,12,0
1769362,42801859811568001,36075032195324526,57226385167357612,0,0,224.70,2,1,1,34,...,1492,0,0.000,0,0.00,0,0,291.8,3,0


## Configure tree two

In [ ]:
params = {
    "max_depth" : 5,
    "eval_metric" : ["mae"],
}

# ENSENBLE MODELS ON VALSET

### Predict - Update - Predict

In [109]:
def gen_predictions(models,labels,num_samples):
    labels = labels[:num_samples].values
    
    predictions = []
    for i in range(len(labels)):
        

        line_prediction = []
        for model in models:
            data = model['data']
            line = xgb.DMatrix(data.iloc[i],feature_names=data.columns)
            predictor = model['predictor']
            prediction = predictor.predict(line)
            line_prediction.append(prediction)
            if i %1000  ==0 :
                print("model",prediction)
                
                
        avg_prediction = 1.0*sum(line_prediction)/len(line_prediction)
        if i % 1000 == 0: 
            print("avg",avg_prediction)
            print("label",labels[i])
            print(i)
        predictions.append(avg_prediction)

    return predictions
            

In [101]:
def mae(predictions,labels):
    result = []
    labels = labels.values
    for i in range(len(predictions)):
        p_i = predictions[i]
        l_i = labels[i]
        diff = abs(p_i-l_i)
        result.append(diff)
    return sum(result)/len(result)

### Train Model One

In [41]:
iters = 20

tree_one = xgb.train(params, trainset_one_dmatrix, evals=[(trainset_one_dmatrix, "train"),(valset_one_dmatrix, 'val')], num_boost_round = iters)

[15:54:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[0]	train-mae:0.201145	val-mae:0.201243
[15:54:56] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[1]	train-mae:0.156018	val-mae:0.156105
[15:55:05] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[2]	train-mae:0.126436	val-mae:0.126491
[15:55:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3]	train-mae:0.10623	val-mae:0.106267
[15:55:24] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[4]	train-mae:0.0929	val-mae:0.092923
[15:55:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[5]	train-mae:0.085222	val-mae:0.085213
[15:55:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 prune

### Train Model Two

In [58]:
iters = 20 

tree_two = xgb.train(params, trainset_two_dmatrix, evals=[(trainset_two_dmatrix, "train"),(valset_two_dmatrix, 'val')], num_boost_round = iters)

[16:14:28] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[0]	train-mae:0.201184	val-mae:0.201106
[16:14:46] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[1]	train-mae:0.156064	val-mae:0.155975
[16:14:57] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[2]	train-mae:0.126485	val-mae:0.126402
[16:15:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[3]	train-mae:0.1061	val-mae:0.106031
[16:15:21] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[4]	train-mae:0.092913	val-mae:0.092844
[16:15:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 60 extra nodes, 0 pruned nodes, max_depth=5
[5]	train-mae:0.085304	val-mae:0.085242
[16:15:44] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 prun

### Ensemble model predictions

In [110]:
models = []
models.append({'data':x_one_val,
                'predictor':tree_one})

models.append({'data':x_two_val,
                'predictor':tree_two})

labels = y_one_val

num_samples = 100000

predictions = gen_predictions(models,labels,num_samples)


('model', array([0.12211746], dtype=float32))
('model', array([0.7672645], dtype=float32))
('avg', array([0.44469097], dtype=float32))
('label', 0.1034)
0
('model', array([0.76649094], dtype=float32))
('model', array([0.4971879], dtype=float32))
('avg', array([0.6318394], dtype=float32))
('label', 0.8519)
1000
('model', array([0.9222779], dtype=float32))
('model', array([0.01415187], dtype=float32))
('avg', array([0.4682149], dtype=float32))
('label', 1.0)
2000
('model', array([0.8417976], dtype=float32))
('model', array([0.8644389], dtype=float32))
('avg', array([0.85311824], dtype=float32))
('label', 0.7659999999999999)
3000
('model', array([0.88109463], dtype=float32))
('model', array([0.9018289], dtype=float32))
('avg', array([0.89146173], dtype=float32))
('label', 0.7917)
4000
('model', array([0.2755858], dtype=float32))
('model', array([0.08915505], dtype=float32))
('avg', array([0.18237042], dtype=float32))
('label', 0.1111)
5000
('model', array([0.3131979], dtype=float32))
('mo

('model', array([0.15701142], dtype=float32))
('model', array([0.5472296], dtype=float32))
('avg', array([0.35212052], dtype=float32))
('label', 0.1481)
52000
('model', array([0.22117701], dtype=float32))
('model', array([0.35415357], dtype=float32))
('avg', array([0.2876653], dtype=float32))
('label', 0.2069)
53000
('model', array([0.6101549], dtype=float32))
('model', array([0.73590076], dtype=float32))
('avg', array([0.6730279], dtype=float32))
('label', 0.6458)
54000
('model', array([0.52700937], dtype=float32))
('model', array([0.05599734], dtype=float32))
('avg', array([0.29150337], dtype=float32))
('label', 0.5385)
55000
('model', array([0.8343046], dtype=float32))
('model', array([0.21457994], dtype=float32))
('avg', array([0.52444226], dtype=float32))
('label', 0.7097)
56000
('model', array([0.08569774], dtype=float32))
('model', array([0.8375398], dtype=float32))
('avg', array([0.46161878], dtype=float32))
('label', 0.0769)
57000
('model', array([0.29311687], dtype=float32))


In [106]:
print(mae(predictions,labels))

[0.1854607]
